># DNN Classifier로 간단하게 학습하기

- 이전 챕터에서 구축하였던 심층 신경망 DNN<sub>(Deep Neural Network)</sub>을 간단하게 구현 (`DNNClassifier` + `SKCompat`)
- *local environment 사용*

In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.learn.python import SKCompat

- 데이터는 wdbc 사용

In [9]:
dataset = pd.read_csv('data/wdbc.csv')

dataset['diagnostic'] = dataset['diagnostic'].apply(lambda X: 0 if X=='M' else 1)
dataset.drop('index', axis = 1, inplace = True)
X_dataset = dataset.drop('diagnostic', axis = 1).values
y_dataset = dataset['diagnostic'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset,
                                                   test_size = 0.2, 
                                                   random_state = 42)

- 전결합(fully-connected)층만으로 구성된 신경망이라면 TensorFlow의 DNN Classifier를 사용하여 간단하게 구축 가능  
  (`tf.contrib.learn.DNNClassifier`)
- 매개변수 `feature_columns`로 특징량 수를 정의하며, 이때 `[tf.contrib.layer.real_valued_column(dimension = 특징량)]`으로 정의
- 매개변수 `hidden_units`로 layer와 unit 수를 정의 (리스트 길이 = layer 수, 리스트 값 = unit 수)
- 매개변수 `n_classes`로 레이블 수 정의
- 매개변수 `model_dir`로 학습 경과와 이후 챕터에서 언급할 TensorBoard로 모니터링 할 수 있게 해 주는 정보들 출력  
  (로컬 경로 외 GCS 경로도 지정 가능 `model_dir = "gs://<BUCKET>/dnnmodel/"`)


In [ ]:
# 특징량 수 정의
feature_columns = [tf.contrib.layers.real_valued_column("", dimension = 30)]

# 은닉층 2개(4개유닛)
classifier = tf.contrib.learn.DNNClassifier(feature_columns = feature_columns,
                                           hidden_units = [4, 4],
                                           n_classes = 2,
                                           model_dir = "./dnnmodel/")

- 구축한 모델은 `SKCompat`으로 변환 가능하며, 이후 scikit-learn 라이브러리에서 사용할 수 있게 됨
- `fit`이라던지 `score` 기능 활용하여 학습 및 평가 가능

In [11]:
# scikit-learn와 비슷한 SKCompat으로 변환
classifier = SKCompat(classifier)

# 학습하기
classifier.fit(x = X_train, y = y_train, steps = 1000, batch_size = 50)

# 정답률 확인
classifier.score(X_test, y_test)

Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023A04FC05F8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5

{'loss': 0.13138488,
 'accuracy': 0.9385965,
 'labels/prediction_mean': 0.6620498,
 'labels/actual_label_mean': 0.622807,
 'accuracy/baseline_label_mean': 0.622807,
 'auc': 0.99443173,
 'auc_precision_recall': 0.9965906,
 'accuracy/threshold_0.500000_mean': 0.9385965,
 'precision/positive_threshold_0.500000_mean': 0.9102564,
 'recall/positive_threshold_0.500000_mean': 1.0,
 'global_step': 1000}